In [2]:
%autosave 0

Autosave disabled


In [3]:
%pip install git+https://github.com/jvkersch/pyconcorde

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jvkersch/pyconcorde to /tmp/pip-req-build-3du3r5y6
  Running command git clone --filter=blob:none --quiet https://github.com/jvkersch/pyconcorde /tmp/pip-req-build-3du3r5y6
  Resolved https://github.com/jvkersch/pyconcorde to commit 71232a8043a3ca79a7a6da2f48e9d89d81d37163
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyconcorde: filename=pyconcorde-0.1.0-cp310-cp310-linux_x86_64.whl size=2494629 sha256=20e8e3359a4f555a33f1b4a3d053c9ddf459fddb41090a630ec16ef74057485a
  Stored in directory: /tmp/pip-ephem-wheel-cache-vs_on7uo/wheels/be/9f/eb/e4f95e06e62f057a045679e4940e536d1b251d1ab4859c6dcc
Successfully built pyconcorde


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# install tweak file
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_class.py' '/content'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/test.py' '/content'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_to_replace/tsp.py' '/usr/local/lib/python3.10/dist-packages/concorde'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_to_replace/util.py' '/usr/local/lib/python3.10/dist-packages/concorde'

In [9]:

import os
import pandas as pd
import numpy as np
import re
import collections
import pickle
from ast import literal_eval
import random
import scipy
import math
from concorde_class import Concorde

#Define Path of Files

Mounted at /content/drive


In [10]:
# path for the 'Data' folder provided by DP
dir_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data'

#Functions

In [ ]:
# generate workflow for file storage /  instance creation

## Volume Matrix Manipulation and Poisson Instances Generation


In [11]:
# create dataframes for a region everyday

def generate_region_volume(region_name):
  # path of folder
  dir_region_instances = dir_loc + '/Instances/' + region_name
  dir_region_volume = dir_loc + '/Volumes/'
  # dir_region_districts = dir_region_instances + '/Districts'

  # load post_object-to-route_pos_id and post_point-to-post_object files 
  po_file = pd.read_csv(dir_region_instances + '/post_order_id_mapping.dat', sep='\t', names=('PostObjectId', 'RoutePosID'))
  pp_file = pd.read_csv(dir_region_instances + '/post_point_information.dat', sep='\t', names=('PostPointId', 'PostObjectId'))

  # pp_file adjustment by splitting list of post object ids
  pp_file['PostObjectId'] = pp_file['PostObjectId'].apply(literal_eval)
  pp_file = pp_file.explode('PostObjectId', ignore_index=True)

  # complete  post object list-district mapping
  region_district_df = generate_region_district(region_name = region_name)
  region_district_df.rename(columns = {'PostObjectID' : 'PostObjectId'}, inplace = True)

  # list of volume file paths
  day_names = []
  vol_path_list = []
  vol_day_map = {}
  day_map = {'mo' : 'Monday',
             'di' : 'Tuesday',
             'mi' : 'Wednesday',
             'do' : 'Thursday',
             'fr' : 'Friday',
             'sa' : 'Saturday'}

  for filename in os.listdir(dir_region_volume):
    vol_path_list.append(dir_region_volume + filename)
    day = filename[-6:-4]
    vol_day_map[filename] = day_map[day]

  # store dataframes of a region, map with post object id
  region_vol_day_dict = {}
  for vol_path in vol_path_list:
    vol_df = pd.read_csv(vol_path, sep = ';')
    vol_df.rename(columns = {'BRIEFE' : 'LETTERS',
                             'PAKETE' : 'PACKAGES',
                             'SONSTIGE' : 'OTHERS',
                             'ROUTEPOS_ID' : 'RoutePosID'},
                  inplace = True)
    
    # combining files to a complete table for a region
    vol_po_df = pd.merge(po_file, vol_df, on = 'RoutePosID', how = 'left')
    vol_po_df = pd.merge(pp_file, vol_po_df, on='PostObjectId', how='right')
    vol_po_df = pd.merge(region_district_df, vol_po_df, on='PostObjectId', how='right')

    # store dataframes in dict
    day_key = vol_day_map[vol_path[-18:]]
    region_vol_day_dict[day_key] = vol_po_df
    
  return region_vol_day_dict

In [12]:
def generate_region_district(region_name):
  district_path = dir_region_instances = dir_loc + '/Instances/' + region_name + '/Districts'
  region_districts_list = []

  for filename in os.listdir(district_path):
    file_var = re.sub('.dat', '', filename)
    file_district = pd.read_csv(district_path + '/' + filename, sep='\t', skiprows = [0,1],
                                names=('PostObjectID', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5')
                                # usecols = [0]
                                )
    file_district['district'] = file_var
    region_districts_list.append(file_district)

  region_districts_df = pd.concat(region_districts_list, ignore_index = True)
  region_districts_df.drop(['dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5'], axis = 1, inplace = True)
  
  return region_districts_df

In [47]:
# generate instances

def generate_instances(region_vol_day, scenario_type, scenario_method, scenario_number, growth_factor):
  # df = region_vol_day.copy() #.copy() used to avoid recopying on the original dataframe
  df = region_vol_day

  sce_letters = 'scenario_' + str(scenario_number) + '_letter'
  sce_packages = 'scenario_' + str(scenario_number) + '_package'
  sce_others = 'scenario_' + str(scenario_number) + '_others'

  df[sce_letters] = df['LETTERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor ))
  df[sce_packages] = df['PACKAGES'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor))
  df[sce_others] = df['OTHERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor))

  sce_all = 'scenario_' + str(scenario_number) + '_all'
  df[sce_all] = df[sce_letters] + df[sce_packages] + df[sce_others]

  return df

In [48]:
# generate real instances based on poisson

def random_poisson_instances(pos_delivery, method, rate_pct):
  rate = rate_pct/100
  rng = np.random.default_rng()
  poisson_dist = rng.poisson(lam = pos_delivery * (1 + rate), size = 52)

  if method == 'random':
    return random.choice(poisson_dist)
  if method == 'mode':
    return scipy.stats.mode(poisson_dist, keepdims = True)[0][0]
  else:
    return 'only options : [random, mode]'

In [32]:
# get start and end node for a district

def start_end_points(region_name, district, df_day):
    df_day_district = df_day[df_day['district'] == district]

    # get start and end node
    district_path = dir_loc + '/Instances/' + region_name + '/Districts/' + district + '.dat'
    rows_needed = [1]
    file_district = pd.read_csv(district_path, sep='\t', skiprows = lambda x : x not in rows_needed,
                                names=('dum_0', 'start_point', 'end_point', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5', 'dum_6')
                                )
    start_point = file_district['start_point'][0]
    end_point = file_district['end_point'][0]

    points = [start_point, end_point]

    return points

In [38]:
# generate distance matrix and mapping for a scenario

def generate_distance_matrix_map(region_name, df_day_district, points, sce_col, district):
      scenario = sce_col

      col_use = ['PostPointId']
      col_use.append(scenario)
      df_day_district_scenario = df_day_district[col_use]
      
      # removing nodes with zero demand
      df_day_district_scenario_filtered = df_day_district_scenario[df_day_district_scenario[scenario] != 0]
      pp_id_day_district_scenario = df_day_district_scenario_filtered['PostPointId'].unique().tolist()
      
      # add start and end node if not in node list yet
      for point in points:
        if point not in pp_id_day_district_scenario:
          pp_id_day_district_scenario.append(point)

      # get distance file and dataframes
      distance_path = dir_loc + '/Instances/' + region_name + '/distances'
      district_distance = pd.read_csv(distance_path + '/distances_' + district + '.dat',
                                  names=['pp_1', 'pp_2', 'dist']
                                  )
      # remove unused postpoints
      district_distance_filtered = district_distance[(district_distance['pp_1'].isin(pp_id_day_district_scenario)) & (district_distance['pp_2'].isin(pp_id_day_district_scenario))]
      distance_matrix_df = district_distance_filtered.pivot(index = 'pp_1', columns = 'pp_2', values = 'dist')

      # distance matrix
      distance_matrix_array = distance_matrix_df.to_numpy()
      distance_matrix_array.tolist()

      # node mapping
      map_val = list(range(0, len(distance_matrix_df)))
      nodes = distance_matrix_df.index.values.tolist()
      mapping = dict(zip(map_val, nodes))

      return distance_matrix_array, mapping

In [51]:
def create_dm_map(region, data, save_loc, rate):
  i = 1
  dir_region = save_loc + '/' + region
  if not os.path.exists(dir_region):
    os.mkdir(dir_region)

  result_region = {}

  # filter by day
  for day in data.keys():
    result_day = {}
    df_day = data[day]

    # filter by district
    for district in df_day['district'].value_counts().index.tolist():
      result_district = {}
      df_day_district = df_day[df_day['district'] == district]

      scenario_list = df_day_district.columns.tolist()[7:]

      # get start and end node
      points = start_end_points(region_name = region, district = district, df_day = df_day)
      
      # filter by scenario
      for scenario in scenario_list:
        # dir of mail
        mail_names = ['letter', 'package', 'others', 'all']
        for mail in mail_names:
          if mail in scenario:
            mail_current = mail
        result_mail = {}

        # dir of demand
        demand_types = ['low', 'medium', 'high']
        for demand in demand_types:
          if demand in scenario:
            demand_current = demand
        result_demand = {}

        # generate mapping and distance matrix
        distance_matrix, mapping = generate_distance_matrix_map(region_name = region, df_day_district = df_day_district,
                                                                points = points, sce_col = scenario, district = district)
        distance_matrix_name = 'dm_' + region + '_' + day + '_' + district + '_' + scenario + '_' + str(rate)
        mapping_name = 'map_' + region + '_' + day + '_' + district + '_' + scenario + '_' + str(rate)
        pickle.dump(distance_matrix, open(dir_region + '/' + "%s.p"%distance_matrix_name, "wb"))
        pickle.dump(mapping, open(dir_region + '/' + "%s.p"%mapping_name, "wb"))

        # log update
        print(i, ':', distance_matrix_name, ':', distance_matrix.shape)
        i = i+1

        result_end = {}
        result_end['dm'] = distance_matrix
        result_end['map'] = mapping_name

        result_demand[demand] = result_end
        result_mail[mail] = result_demand[demand]
      result_district[district] = result_mail
    result_day[day] = result_district 
  result_region[day] = result_day

  return result_region

In [19]:
# exact solution with concord

def concord_solver(distance_matrix):
  concorde = Concorde()
  concorde_sol = concorde.run_ProbATSP(distance_matrix)
  return concorde.route, concorde.optimal_value

#Process Examples (Using Functions)

In [52]:
%%time
# ['Warmsen', 'Uerze', 'Hannover 92']

region_vol_day_dict = generate_region_volume(region_name = "Warmsen")

CPU times: user 11.5 s, sys: 1.06 s, total: 12.6 s
Wall time: 13.7 s


In [21]:
region_vol_day_dict['Monday'].head()

,PostObjectId,district,PostPointId,RoutePosID,LETTERS,PACKAGES,OTHERS
0,16,31606-14,16,AD14E08623FF9F5AD2293E7DEDB4F4B1,0.45,0.293333,0.0
1,17,31606-14,17,F48CB0CAB0BC117AE030007F0100574C,0.45,0.293333,0.0
2,18,31606-14,16,F48CB0CAB0C7117AE030007F0100574C,0.45,0.293333,0.0
3,19,31606-14,18,F48CB0CAB0C8117AE030007F0100574C,0.00,0.000000,0.0
4,20,31606-14,19,069AE286923C1800E040400A09131416,0.00,0.000000,0.0


In [53]:
%%time
#Creating the complete dictionary of volumes for a region
warmsen_instances = {}

#INPUT Rate of mail change 
# rate in percentage
rate = 50
rate_mapping = {'low' : -rate, 'medium' : 0, 'high' : rate}

# create instances for one region
for key in region_vol_day_dict.keys():
  # create instances per day
  for key_rate in rate_mapping:
    dummy = generate_instances(region_vol_day = region_vol_day_dict[key],
                               scenario_type = random_poisson_instances,
                               scenario_method = 'random',
                               scenario_number = key_rate,
                               growth_factor = rate_mapping[key_rate])
  warmsen_instances[key] = dummy

CPU times: user 18.1 s, sys: 343 ms, total: 18.5 s
Wall time: 20.4 s


In [54]:
warmsen_instances['Tuesday'].head()

,PostObjectId,district,PostPointId,RoutePosID,LETTERS,PACKAGES,OTHERS,scenario_low_letter,scenario_low_package,scenario_low_others,scenario_low_all,scenario_medium_letter,scenario_medium_package,scenario_medium_others,scenario_medium_all,scenario_high_letter,scenario_high_package,scenario_high_others,scenario_high_all
0,16,31606-14,16,AD14E08623FF9F5AD2293E7DEDB4F4B1,5.796667,0.69,0.273333,3,0,0,3,5,0,0,5,10,1,0,11
1,17,31606-14,17,F48CB0CAB0BC117AE030007F0100574C,5.796667,0.69,0.273333,2,0,0,2,1,0,2,3,11,0,0,11
2,18,31606-14,16,F48CB0CAB0C7117AE030007F0100574C,5.796667,0.69,0.273333,3,1,2,6,5,0,0,5,5,1,0,6
3,19,31606-14,18,F48CB0CAB0C8117AE030007F0100574C,0.000000,0.00,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
4,20,31606-14,19,069AE286923C1800E040400A09131416,0.000000,0.00,0.000000,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
print(warmsen_instances['Tuesday']['scenario_low_all'].sum())
print(warmsen_instances['Tuesday']['scenario_medium_all'].sum())
print(warmsen_instances['Tuesday']['scenario_high_all'].sum())

4593
9279
13750


In [56]:
z = warmsen_instances['Tuesday']


print(z[z['district'] == '31606-14']['scenario_low_all'].sum())
print(z[z['district'] == '31606-14']['scenario_medium_all'].sum())
print(z[z['district'] == '31606-14']['scenario_high_all'].sum())

262
566
859


In [57]:
%%time

# growth_rate_map_50 = {'high' : 'inc_50', 'low' : 'dec_50', 'medium' : 'base'}

result_region = create_dm_map(region = 'Warmsen', data = warmsen_instances,
                              save_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/',
                              rate = rate)

1 : dm_Warmsen_Tuesday_31603-06_scenario_low_letter_50 : (159, 159)
2 : dm_Warmsen_Tuesday_31603-06_scenario_low_package_50 : (42, 42)
3 : dm_Warmsen_Tuesday_31603-06_scenario_low_others_50 : (113, 113)
4 : dm_Warmsen_Tuesday_31603-06_scenario_low_all_50 : (239, 239)
5 : dm_Warmsen_Tuesday_31603-06_scenario_medium_letter_50 : (237, 237)
6 : dm_Warmsen_Tuesday_31603-06_scenario_medium_package_50 : (67, 67)
7 : dm_Warmsen_Tuesday_31603-06_scenario_medium_others_50 : (188, 188)
8 : dm_Warmsen_Tuesday_31603-06_scenario_medium_all_50 : (329, 329)
9 : dm_Warmsen_Tuesday_31603-06_scenario_high_letter_50 : (304, 304)
10 : dm_Warmsen_Tuesday_31603-06_scenario_high_package_50 : (92, 92)
11 : dm_Warmsen_Tuesday_31603-06_scenario_high_others_50 : (230, 230)
12 : dm_Warmsen_Tuesday_31603-06_scenario_high_all_50 : (364, 364)
13 : dm_Warmsen_Tuesday_31600-01_scenario_low_letter_50 : (140, 140)
14 : dm_Warmsen_Tuesday_31600-01_scenario_low_package_50 : (36, 36)
15 : dm_Warmsen_Tuesday_31600-01_scenari

In [ ]:
# change akshay's output into list of numbers